In [42]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from ta.trend import SMAIndicator
from ta.volatility import BollingerBands
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
df=pd.DataFrame()
dfout=pd.DataFrame()
start_date="2000-10-01"
end_date="2021-10-01"
txt = "{:%}"
wind=14
for i in range(0,5):

    stk=sip[i]
    #df = yf.download(stk, start_date, end_date,interval=intr).dropna()

    dfd = yf.download(stk, start_date, end_date,interval='1d').dropna()
    dfd['Close_1']=dfd['Close'].shift(1)
    dfd['Volume_1']=dfd['Volume'].shift(1)


    dfd['efid']=ForceIndexIndicator(close=dfd.Close_1,volume=dfd.Volume_1,window=wind,fillna=True).force_index()
    dfd['efi_rsid']=RSIIndicator(close=dfd.efid,window=wind,fillna=True).rsi()
    dfd["kd"]=StochRSIIndicator(close=dfd.Close_1,window=wind,smooth1=3, smooth2=3).stochrsi_k() 
    dfd['bbh']=BollingerBands(close=dfd.Close_1,window=20,window_dev=2,fillna=True).bollinger_hband()
    dfd['bbm']=BollingerBands(close=dfd.Close_1,window=20,window_dev=2,fillna=True).bollinger_mavg()
    dfd['bbl']=BollingerBands(close=dfd.Close_1,window=20,window_dev=2,fillna=True).bollinger_lband()

    dfd['sma200']=SMAIndicator(close=dfd.Close_1,window=200,fillna=True).sma_indicator()
    dfd['sma50']=SMAIndicator(close=dfd.Close_1,window=50,fillna=True).sma_indicator()
    dfd['sma20']=SMAIndicator(close=dfd.Close_1,window=20,fillna=True).sma_indicator()
    dfd['sma10']=SMAIndicator(close=dfd.Close_1,window=10,fillna=True).sma_indicator()
    dfd["Chng"]=(dfd.Close-dfd.Open)
    dfd['ChngOv']=dfd.Close-dfd.Close_1
    dfd['Chng_1']=dfd.Chng.shift(1)
    #dfd['Chng_smaf']=SMAIndicator(close=dfd.Chng_1,window=5,fillna=True).sma_indicator()
    #dfd['Chng_smas']=SMAIndicator(close=dfd.Chng_1,window=20,fillna=True).sma_indicator()
    #dfd['Vol_smaf']=SMAIndicator(close=dfd.Volume_1,window=5,fillna=True).sma_indicator()
    #dfd['Vol_smas']=SMAIndicator(close=dfd.Volume_1,window=20,fillna=True).sma_indicator()
    #print(dfd.tail)
    dfd=dfd.dropna()
    perd=30

    conditions = [ 
    (dfd['Open']>0)
    #&(dfd['Close_1']<dfd.bbl)
    #&(dfd['Close_1']<dfd.bbm-dfd.bbg)    
    &(dfd['kd']>np.percentile(dfd['kd'],perd)) 
    #&(dfd['kd']>=0)
    #&(dfd['kd']<0.3)
    #(dfd['efi_rsid']<np.percentile(dfd['efi_rsid'],100-perd))  
    &(dfd['efi_rsid']<np.percentile(dfd['efi_rsid'],perd))  
    &(dfd['sma10']<dfd['sma200'])


    #&(dfd['Chng_smaf']>dfd['Chng_smas'])
    #&(dfd['Vol_smaf']>dfd['Vol_smas'])
    #&(dfd['sma20']<dfd['sma50'])
    #&(dfd['sma10']>dfd['sma20'])
    #&(dfd['Open']<dfd['sma10'])
    #(dfd['Open']>dfd['Close_1'])
    #& (dfd['Close_1']>=dfd['psma'])
    ,
    (dfd['Open']>0)
    &(dfd['kd']<40)   

    ]

    choices = [
    1,-1
    ]
    dfd['trade_pass']=np.select(conditions,choices,default=0)
    dfd_long=dfd[dfd['trade_pass']==1]
    #print(dfd_long)
    long_prof=np.sum(dfd_long.ChngOv)
    trade_rate=len(dfd_long.index)/(len(dfd.index))
    hold_wr=len(dfd[dfd.ChngOv>0])/(len(dfd.index))
    long_wr=len(dfd_long[dfd_long.ChngOv>0])/(len(dfd_long.index))
    hold=np.sum(dfd.ChngOv)
    #print(hold)
    #print(long_prof)
    #print(trade_rate)
    #print(hold_wr)
    #print(trade_wr)
    #print(max(dfd.kd))
    temp=pd.Series([stk,hold,hold_wr,long_prof,long_wr,trade_rate])
    dfout=dfout.append(temp,ignore_index=True)

print(dfout)
print(np.average(dfout.iloc[:,2]))
print(np.average(dfout.iloc[:,4]))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
      0           1         2          3         4         5
0   MMM  126.763748  0.519886  96.929947  0.676768  0.018839
1   ABT   94.562702  0.510371  22.144966  0.594595  0.014082
2  ABBV   72.020004  0.533793   3.910023  0.620000  0.022989
3  ABMD  298.394989  0.505994  11.695015  0.570093  0.020362
4   A